## Setting up Envoirnment

In [2]:
import os
os.environ["GROQ_API_KEY"] = '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'

In [4]:
!pip install -qU langchain langchain-groq

In [26]:
from langchain_groq import ChatGroq
from typing import List
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator

## Initializing LLM

In [5]:
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=3,
    # other params...
)

## Definining the Agent

As we need to extract the types of entities and related predicates so we are using the Pydantic Model for structure output.

In [11]:
# Define your desired data structure.
class TriplexModel(BaseModel):
    entity_name_list: List[str] = Field(description="The list of all entities type in text")
    predicates_list: List[str] = Field(description="The list of all predicate type in text")


In [12]:
parser = PydanticOutputParser(pydantic_object=TriplexModel)

Prompt Engneering comes here. We can write a better prompt but for me this is working as expected

In [19]:
prompt = PromptTemplate(
    template="You will be given a text.  You need to extract the list of entities types and predicates type . \n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

Input text comes here;

In [20]:
text = """
Deeper neural networks are more difficult to train. We present a residual learning framework to ease the training of networks that are substantially deeper than those used previously. We explicitly reformulate the layers as learning residual functions with reference to the layer inputs, instead of learning unreferenced functions. We provide comprehensive empirical evidence showing that these residual networks are easier to optimize, and can gain accuracy from considerably increased depth. On the ImageNet dataset we evaluate residual nets with a depth of up to 152 layers---8x deeper than VGG nets but still having lower complexity. An ensemble of these residual nets achieves 3.57% error on the ImageNet test set. This result won the 1st place on the ILSVRC 2015 classification task. We also present analysis on CIFAR-10 with 100 and 1000 layers.
The depth of representations is of central importance for many visual recognition tasks. Solely due to our extremely deep representations, we obtain a 28% relative improvement on the COCO object detection dataset. Deep residual nets are foundations of our submissions to ILSVRC & COCO 2015 competitions, where we also won the 1st places on the tasks of ImageNet detection, ImageNet localization, COCO detection, and COCO segmentation.
 """

Making the Chain and Querying it.

In [22]:
chain = prompt | llm | parser

result = chain.invoke({"query": text})

Thanks | Mohammad Ali Abbas